In [1]:
import numpy as np
import numpy.linalg as npl
import scipy.stats as ss
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import Bounds
from time import time
import os
import threading

cdf_gumbel = lambda t,mu,beta : np.exp(-np.exp(-(t-mu)/beta))

cdf_gumbel_inv = lambda s,mu,beta : mu - beta * np.log(-np.log(s))

cdf_norm = lambda t,sigma : ss.norm.cdf(t/sigma)

In [2]:
# 0 : metro
# 1 : caminata
# 2 : auto
# X : matrice NxM, N individuos, M modos de transporte.                       

def prob_new2(B0,B,X,D_auto,C,alpha,m):
    
    
    
    term_metro =  np.exp(B0[0]+B[0]*X[:,0]+B[1]*C[:,0]).reshape((-1,1))
    term_caminata =  np.exp(B0[1]+B[0]*X[:,1]+B[1]*C[:,1]).reshape((-1,1))
    term_auto = np.exp(B[0]*alpha*D_auto+B[1]*C[:,2]).reshape((-1,1))
    
    terms = np.concatenate([term_metro,term_caminata,term_auto],axis = 1)
    
    result = terms[:,m]/(np.sum(terms,axis = 1))
    return result

def log_verosimilitud_new2(B_all,X,D_auto,C,alpha,mode_indice):
    
    S = 0
    
    B0 = B_all[:2]
    
    B = B_all[2:]
    
    for i in range(X.shape[0]):
        
        S += np.log(prob_new2(B0,B,X[i,:].reshape((1,-1)),D_auto[i].reshape((1,1)),\
                             C[i,:].reshape((1,-1)),alpha,mode_indice[i]))
        
    return -float(S)         

In [3]:
def prob_old(B0,B,X,C,m,eta=np.array([None])):
    
    B00 = [B0[0],B0[1],0]
    
    if eta[0] == None:   
    
     result = np.exp(B00[m]+B[0]*X[:,m]+B[1]*C[:,m])/(np.sum(np.exp(B00+B[0]*X+B[1]*C),axis = 1))
     
    else: 
        
     eta_new = np.concatenate([np.zeros((X.shape[0],2)),eta],axis = 1) 
        
     result = np.exp(B00[m]+B[0]*X[:,m]+B[1]*C[:,m]+B[2]*eta_new[:,m])\
    /(np.sum(np.exp(B00+B[0]*X+B[1]*C+B[2]*eta_new),axis = 1))   

    return result

def log_verosimilitud_old(B_all,X,C,mode_indice,eta=np.array([None])):
    
    S = 0
    
    B0 = B_all[:2]
    
    B = B_all[2:]
    
    if eta[0] == None:
        
        for i in range(X.shape[0]):

            S += np.log(prob_old(B0,B,X[i,:].reshape((1,-1)),C[i,:].reshape((1,-1))\
                                 ,mode_indice[i]))
            
    else:        
        
        for i in range(X.shape[0]):
        
            S += np.log(prob_old(B0,B,X[i,:].reshape((1,-1)),C[i,:].reshape((1,-1)),\
                                 mode_indice[i],eta[i,:].reshape((1,-1))))
        
    return -float(S)  

def moments(B_all,X,D_auto,C,mode_indice,alpha):
    
    result = []
    
    B0 = [B_all[0],B_all[1],0]
    
    B = B_all[2:]
    
    S = 0
    
#Ecuación metro
    for i in range(X.shape[0]): 
        
        S +=(int(mode_indice[i] == 0)-prob_old(B0,B,X[i,:].reshape((1,-1)),C[i,:].reshape((1,-1)),0))
        
    result.append(S)
    
    S = 0
    
#Ecuación caminata
    for i in range(X.shape[0]): 
        
        S +=(int(mode_indice[i] == 1)-prob_old(B0,B,X[i,:].reshape((1,-1)),C[i,:].reshape((1,-1)),1))
        
    result.append(S)    

#Ecuación suma tiempo/distancia

    S = 0

    for i in range(X.shape[0]):
      for m in range(2):  
    
        S += (int(mode_indice[i] == m)-prob_old(B0,B,X[i,:].reshape((1,-1)),\
                                                C[i,:].reshape((1,-1)),m))*X[i,m]
        
      S +=  alpha*(int(mode_indice[i] == 2)-prob_old(B0,B,X[i,:].reshape((1,-1)),\
                                                     C[i,:].reshape((1,-1)),2))*D_auto[i] 
    
    result.append(S)
    
#Ecuación suma costo

    S = 0

    for i in range(X.shape[0]):
      for m in range(3):  
    
        S += (int(mode_indice[i] == m)-prob_old(B0,B,X[i,:].reshape((1,-1)),\
                                                C[i,:].reshape((1,-1)),m))*C[i,m]
    
    result.append(S)    
    
    return (1/X.shape[0])*np.array(result).reshape((4,))

In [4]:
from scipy.optimize import fsolve

In [5]:
a_metro = 35**(-1)
a_caminata = 3**(-1)
a_auto = 25**(-1)


beta1 = -0.02   # Sensitivity to time travel
beta2 = -0.04   # Sensitivity to cost travel

beta0_metro = 0.7
beta0_caminata = 0.4
beta0_auto = 0

theta = 0.7
sigma_eta = 0.5

In [6]:
sigma_u = np.sqrt(1 + beta1**2*sigma_eta**2 + 2 * beta1 * sigma_eta * theta)

beta_g = 1

mu_g = 0

In [11]:
tramo_metro = lambda dist,init_price=1,largo_tramo = 10,factor=0.5 : \
init_price + factor * np.int64(dist/largo_tramo)

epochs = 500

n = 500

file_exists = os.path.exists(f'{n}_individuos_{epochs}_simulaciones_2_variables_all_inter.xlsx')

list_corrcoef = []

B_all_moments = []

B_all_2S = []

B_all_old = []

B_all_CF = []

list_moments = []

tI = time()

lamb = 0.45

columns = ['beta0_metro_moments','beta0_caminata_moments','beta0_auto_moments','beta1_moments',\
         'beta2_moments','beta0_metro_2S','beta0_caminata_2S','beta0_auto_2S','beta1_2S',\
         'beta2_2S','beta0_metro_old','beta0_caminata_old','beta0_auto_old','beta1_old',\
         'beta2_old','beta0_metro_CF','beta0_caminata_CF','beta0_auto_CF','beta1_CF',\
         'beta2_CF','gamma_CF','sigma_eta_est','corr_eta_eps']

if file_exists:
    
    df_inter = pd.read_excel(f'{n}_simulaciones_2_variables_all_inter.xlsx',index_col = 0)
    
    epoch = len(df_inter) 
    
else:
    
    epoch = 0
    
    df_inter = pd.DataFrame(columns = columns,data = [])
    
print(epoch)    
  
while epoch <= epochs: 
    
    epoch += 1   
    
    if epoch%10 == 0:
        
        print('************',epoch,'simulaciones')
    
    v_metro = np.random.randn(n)
    v_caminata= np.random.randn(n)
    v_auto = np.random.randn(n)
    eta_auto = sigma_eta * np.random.randn(n)
    
    D_metro = 2+28*np.random.rand(n)
    
    D_caminata = D_metro + np.random.randn(n)
    
    D_auto = D_metro + np.random.randn(n)
  
    X_metro = a_metro * D_metro + v_metro 
    
    X_caminata = a_caminata * D_caminata + 0.1*v_caminata
  
    X_auto = a_auto * D_auto + v_auto + eta_auto
    
    X = np.concatenate([X_metro.reshape( (-1,1)),X_caminata.reshape((-1,1)),\
                    X_auto.reshape(-1,1)],axis = 1)    
    
    XD = np.concatenate([X_metro.reshape( (-1,1)),X_caminata.reshape((-1,1)),\
                    D_auto.reshape(-1,1)],axis = 1)
    
    C_metro = tramo_metro(D_metro)
    
    C_auto = 0.2 * D_auto
    
    C_caminata = np.zeros(C_auto.shape)
    
    C = np.concatenate([C_metro.reshape( (-1,1)),C_caminata.reshape((-1,1)),\
                    C_auto.reshape(-1,1)],axis = 1)     
    
    if sigma_eta >0:
     u = cdf_gumbel_inv(cdf_norm(eta_auto + lamb * np.random.randn(n),\
                                 np.sqrt(sigma_eta**2+lamb**2)),0,1)
    else:
     u =  cdf_gumbel_inv(np.random.rand(n),0,1)
    
    eps_auto = u - beta1 * eta_auto
    eps_metro = cdf_gumbel_inv(np.random.rand(n),0,1)
    eps_caminata = cdf_gumbel_inv(np.random.rand(n),0,1)
    
    list_corrcoef.append([np.corrcoef(eta_auto,eps_auto)[0,1]])
    
    U_metro = beta0_metro + beta1 * X_metro + beta2 * C_metro + eps_metro 
    U_caminata = beta0_caminata + beta1 * X_caminata + eps_caminata 
    U_auto = beta0_auto + beta1 * X_auto + beta2 * C_auto + eps_auto
    
    mode_indice = np.array(pd.DataFrame({'Metro':U_metro,'Caminata':U_caminata,'Auto':U_auto})\
    .apply(lambda x : np.argmax(x),axis = 1))
    
    alpha_auto_est = np.cov(D_auto,X_auto)[0,1]/np.var(D_auto)
    
    eta_auto_est = X_auto - alpha_auto_est * D_auto
    
    sol_new = minimize(lambda B_all : \
                       log_verosimilitud_new2(B_all,X,D_auto,C,alpha_auto_est,mode_indice),\
                       np.random.randn(4)+[1,1,-1,-1])
    
    B_all_2S.append([sol_new.x[0],sol_new.x[1],0,sol_new.x[2],sol_new.x[3]])        
    
    x_sol = fsolve(lambda B_all : moments(B_all,X,D_auto,C,mode_indice,alpha_auto_est)\
                   ,sol_new.x)
    
    B_all_moments.append([x_sol[0],x_sol[1],0,x_sol[2],x_sol[3]])
    
#    list_moments.append(moments([x_sol[0],x_sol[1],x_sol[2]],X,C,\
#                                D_auto,mode_indice,alpha_auto_est))
    
    sol_old = minimize(lambda B_all : log_verosimilitud_old(B_all,X,C,mode_indice),\
                       [beta0_metro,beta0_caminata,beta1,beta2])
    
    B_all_old.append([sol_old.x[0],sol_old.x[1],0,sol_old.x[2],sol_old.x[3]])  
    
    sol_CF = minimize(lambda B_all : log_verosimilitud_old(B_all,X,C,mode_indice,\
                eta_auto_est.reshape((-1,1))),[1,1,-1,-1,2])
    
    B_all_CF.append([sol_CF.x[0],sol_CF.x[1],0,sol_CF.x[2],sol_CF.x[3],sol_CF.x[4]\
                     ,np.std(eta_auto_est)])     
    
    if epoch %100 == 0:
            
        B_all = np.concatenate([B_all_moments,B_all_2S,B_all_old,B_all_CF,\
                               list_corrcoef],axis = 1)
            
        df_new_inter = pd.DataFrame(data = B_all , columns = columns)
        
        if file_exists:
            
          df_new_inter = pd.concat([df_inter,df_new_inter],axis = 0)
                
        df_new_inter.to_excel(f'{n}_individuos_{epochs}_simulaciones_2_variables_all_inter.xlsx')  
        
        print('Intermedio guardado')
    
    
    
tF = time()

print('Se demorró',round((tF-tI)/60,0),'minutos.')

0
************ 10 simulaciones
************ 20 simulaciones
************ 30 simulaciones
************ 40 simulaciones
************ 50 simulaciones
************ 60 simulaciones
************ 70 simulaciones
************ 80 simulaciones
************ 90 simulaciones
************ 100 simulaciones
Intermedio guardado
************ 110 simulaciones
************ 120 simulaciones
************ 130 simulaciones
************ 140 simulaciones
************ 150 simulaciones
************ 160 simulaciones
************ 170 simulaciones
************ 180 simulaciones
************ 190 simulaciones
************ 200 simulaciones
Intermedio guardado
************ 210 simulaciones
************ 220 simulaciones
************ 230 simulaciones
************ 240 simulaciones
************ 250 simulaciones
************ 260 simulaciones
************ 270 simulaciones
************ 280 simulaciones
************ 290 simulaciones
************ 300 simulaciones
Intermedio guardado
************ 310 simulaciones
************ 320 

In [8]:
df = pd.read_excel(f'{n}_simulaciones_2_variables_all_inter.xlsx',index_col = 0)

In [9]:
df.shape

(2000, 23)

In [10]:
cols = ['beta0_metro','beta0_caminata','beta0_auto','beta_tiempo','beta_costo']
modelos = ['Moments','2S','VS','CF']

pd.DataFrame(data = np.array(df.mean())[:20].reshape(4,5),columns = cols,index = modelos)

,beta0_metro,beta0_caminata,beta0_auto,beta_tiempo,beta_costo
Moments,0.706403,0.410096,0.0,-0.020603,-0.039271
2S,0.706022,0.409724,0.0,-0.020599,-0.039253
VS,0.837111,0.327001,0.0,0.045649,0.032331
CF,0.768263,0.471172,0.0,-0.020760,-0.040276


In [11]:
df.to_excel(f'{n}_datos_simulaciones_2_variables.xlsx')